#### PIPE - 7:    
This pipe covers the following : 
1. All features of pipe 6
2. Video frame blurring for trackers
3. Integration for front end area selection while maintaining the same distance
5. Integration of MP queue in phase one
6. ANRP Processing phase 2 - Enhance cropped images

#### Processing mask cooredinates
cv2.rectangle(image, (self.camera.boundary.get('x'), self.camera.boundary.get('y')),(self.camera.boundary.get('x2'), self.camera.boundary.get('y2'))

In [1]:
import cv2
import numpy as np
import pandas as pd
import torch
import time
import threading 
from ultralytics import YOLO
from pathlib import Path
from boxmot import StrongSORT
from utils.utils import *
start_time=time.time()
down = {}
up = {}
text_color = (0, 0, 0)  # Black color for text
yellow_color = (0, 255, 255)  # Yellow color for background
red_color = (0, 0, 255)  # Red color for lines
blue_color = (255, 0, 0)  # Blue color for lines

counter_down = []
counter_up = []

first_boundry_y=170

red_line_y = 198

blue_line_y = 280

offset = 6
# Specify the start and end points of the line
start_point = (300, 198)
end_point = (300, 280)

all_id_tracker=[]
voilation_id_tracker=[]
violation_frame_tracker={}
tracker=tracker_init(cuda_device=torch.cuda.is_available(),cuda_device_number=1)
model=YOLO('yolov8s.pt')
video_path='/media/hlink/hd/test_videos/testx_vid_1.mp4'
vid = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_main = cv2.VideoWriter('output_strongs.avi', fourcc, 20.0, (1020, 500))
frame_count=0
all_frames_record_path=create_directory('all_frames_record')
violation_frames_record_path=create_directory('all_violation_record')
x_start=300
x_end=800
y_start=198
y_end=280

while True:    
    ret, frame = vid.read()
    
    if not ret:
        break
    
    resized_image=cv2.resize(frame,(1020, 500))
    blurred_image=blur_except_rectangle(resized_image,x_start,y_start,x_end,y_end,offset_size=[30,80],blur_kernel_size=(15,15))
    
    results=model.predict(blurred_image,conf=0.4,verbose=False,device=[1],classes=[2,3])
    # print(f"\n----------------------------------------------------------------------------------\nResult without detection \n{results[0].boxes.data}\n----------------------------------------------------------------------------------\n")
    blurred_image=line_plotter(frame=blurred_image,x_start=300,x_end=774,y=198,line_color=red_color,line_name='boundry 1',line_thickness=2,text_color=text_color)
    blurred_image=line_plotter(frame=blurred_image,x_start=300,x_end=800,y=280,line_color=blue_color,line_name='boundry 2',line_thickness=2,text_color=text_color)
    
    # Specify the color of the line in BGR format (here, it's white)
    color = (255, 255, 255)
    
    # Draw the vertical line on the image
    thickness = 2  # You can adjust the thickness as needed
    cv2.line(blurred_image, start_point, end_point, color, thickness)
    
    # im=cv2.rectangle(blurred_image, (300, 198), (800, 280), (255, 255, 255), 2)
    
    px,conf=prediction_coordinated_hadler(results)
    dets = []
    # Experimenting
    dets=tracker_element_creator(dets,px,conf)        
    
    # print('out_from')
    
    dets = np.array(dets)
    # print(dets)
    if len(dets) > 0:
        tracks = tracker.update(dets, blurred_image) # --> M X (x, y, x, y, id, conf, cls, ind)
        # print('tracks',tracks)    
        for track in tracks:
                blurred_image,object_id,cx,cy,bbox_list=plot_tracks(track,blurred_image)
                
                # print("vblkjsenjlks",bbox_list)
                all_id_tracker.append(object_id)
                
                # Adding file path for csv
                csv_file_path=detection_coordinate_write(frame_count,object_id,bbox_list,'all_frame_detection_detail.csv')
            
                if red_line_y<(cy+offset) and red_line_y > (cy-offset):
                    # print("entered if 1")
                    down[object_id]=time.time()
            
                if object_id in down:
                    # print("entered if 2")
               
                    if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
                         # print("entered if 3")
                         elapsed_time=time.time() - down[object_id] 
                         # print("entered if 4")
                         if counter_down.count(object_id)==0:
                            counter_down.append(object_id) 
                            distance = 9
                            est_speed=speed_calculator_kmph(distance,elapsed_time)
                            if est_speed>20:
                                print(f'violation_detected : {object_id}')
                                voilation_id_tracker.append(object_id)
                                violation_frame_tracker[object_id]=frame_count
                                # violation_frame_writer_op=frame_writer(frame_count,im,violation_frames_record_path)
                                print("bbox_list-",bbox_list)
                                violation_frame_writer_op=frame_writer(frame_count,resized_image,violation_frames_record_path)
                                violation_frame_writer_op=frame_writer(frame_count,resized_image,violation_frames_record_path,coordiante_blur=True,bbox_list=bbox_list)
                                violation_csv_file_path=detection_coordinate_write(frame_count,object_id,bbox_list,'violation_frame_detection_detail.csv')
                                
                            print(f"for {object_id} speed is -> {est_speed}")
    
    all_frame_writer_op=frame_writer(frame_count,resized_image,all_frames_record_path)    
    frame_count+=1
    out_main.write(resized_image)
    
    # # break on pressing q or spaceq
    cv2.imshow('Strong Sort Detection detection', blurred_image)     
    # key = cv2.waitKey(25) & 0xFF
    
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()
remove_model_from_gpu(model)
end_time=time.time()
print(f'total_time {end_time-start_time}')

2024-05-22 10:50:10.299 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "models/osnet_x0_25_msmt17.pt"


violation_detected : 3
bbox_list- [484, 237, 581, 319]
for 3 speed is -> 42
violation_detected : 8
bbox_list- [463, 223, 569, 329]
for 8 speed is -> 49
violation_detected : 25
bbox_list- [546, 252, 573, 300]
for 25 speed is -> 37
violation_detected : 35
bbox_list- [611, 255, 641, 298]
for 35 speed is -> 32
violation_detected : 50
bbox_list- [538, 253, 567, 297]
for 50 speed is -> 34
violation_detected : 67
bbox_list- [369, 230, 465, 327]
for 67 speed is -> 40
violation_detected : 91
bbox_list- [531, 249, 563, 302]
for 91 speed is -> 32
violation_detected : 96
bbox_list- [356, 222, 452, 329]
for 96 speed is -> 43
violation_detected : 100
bbox_list- [418, 221, 523, 336]
for 100 speed is -> 46
violation_detected : 107
bbox_list- [613, 255, 647, 298]
for 107 speed is -> 29
violation_detected : 144
bbox_list- [346, 238, 435, 322]
for 144 speed is -> 53
violation_detected : 151
bbox_list- [616, 257, 641, 297]
for 151 speed is -> 28
violation_detected : 138
bbox_list- [464, 236, 554, 314]
for

In [2]:
%%time
main_violation_json=voilation_capture_json_creator(voilation_id_tracker,csv_file_path)
maintained=json_frame_order_checker(main_violation_json)
if maintained:
    evidance_img_dir_paths,evidance_clip_dir_paths=evidance_directories_creator(main_violation_json)
    evidance_img_dir_paths=evidance_img_separator(evidance_img_dir_paths,all_frames_record_path,main_violation_json)
    video_clip_creator_mt(voilation_id_tracker,violation_frame_tracker,evidance_clip_dir_paths,all_frames_record_path)
    cropped_images_path_list=evidance_cropper_mt(evidance_img_dir_paths,main_violation_json)
    deblurred_image_paths=deblur_images(cropped_images_path_list,main_violation_json)

NameError: name 'voilation_capture_json_creator' is not defined

In [ ]:
%%time
# Using default 8 threads to achive max effiencicy
enhance_image_path=image_enhancement_using_limit_mpx(deblurred_image_paths,num_threads=8)

In [5]:
%%time
cropped_lp_paths=preprocssing_part_one(enhance_image_path,'models/LPD.pt',2)

Exception in thread Thread-103 (sub_process_for_preprocess_one):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/home/hlink/workspace/learning/boxmot/bmt/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hlink/workspace/learning/boxmot/utils/utils.py", line 778, in sub_process_for_preprocess_one
    idx=i.split('/')[-1]
                     ^^^^
  File "/home/hlink/workspace/learning/boxmot/bmt/lib/python3.11/site-packages/ultralytics/engine/model.py", line 453, in predict
    return self.predictor.predict_cli(source=source) if is_cli else self.predictor(source=source, stream=stream)
                                                                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hlink/workspace/l

CPU times: user 918 ms, sys: 982 ms, total: 1.9 s
Wall time: 1.35 s


In [6]:
enhanced_cropped_images_lp=image_enhancement_using_limit_mpx(cropped_lp_paths,num_threads=8,dir_name='cropped_lp_enhanced')

In [ ]:
def common_chars(input_1, input_2, input_3):
    final_string = ''
    for i in range(min(len(input_1), len(input_2), len(input_3))):
        char_count = {input_1[i]: 1, input_2[i]: 1, input_3[i]: 1}
        if len(char_count) == 1:
            final_string += input_1[i]  # Append any character since all are the same
        elif len(char_count) == 2 and input_2[i] == input_3[i]:
            final_string += input_2[i]  # Append character present in both inputs 2 and 3
        else:
            sorted_chars = sorted(char_count.keys(), key=lambda x: -char_count[x])
            final_string += sorted_chars[0]  # Append the character that appears in more inputs
    return final_string

In [ ]:
def anrp_processor(anrp_folder_paths,yolo_model,img_enhancer_model_path,tr_device,tr_model,tr_processor):
    anrp_detections=[]
    for i in anrp_folder_paths:
        sorted_files=sort_files_by_creation_time(i['main_folder_path'])
        final_json={}
        np_counter={}
        for j in sorted_files:
            final_string,cropped_img_number_plate,upscaled_image,ocr_image,image_name,image_count = anrp(image_path=j,yolo_model=yolo_model,img_enhancer_model_path=img_enhancer_model_path,tr_device=tr_device,tr_model=tr_model,tr_processor=tr_processor)
            if final_string != None:
                    if final_string in np_counter.keys():
                        np_counter[final_string]+=1
                    else:
                        np_counter[final_string]=0
        if len(np_counter)!=0:    
            liscense_plate,detection_number=find_max_key(np_counter) 
            confidence=(detection_number/len(sorted_files))*100
            final_json['count_id']=i['count_id']
            final_json['number_plate_ocr_value']=liscense_plate
            final_json['confidence']=confidence
            final_json['evidance_number_plate_image']=frame_writer(count=image_count,frame=cropped_img_number_plate,folder_path=i['cropped_folder_path'],resize=False)
            final_json['ocr_evidance_number_plate_image']=frame_writer(count=image_count,frame=ocr_image,folder_path=i['ocr_folder_path'],resize=False)
            final_json['upscaled_evidance_number_plate_image']=frame_writer(count=image_count,frame=upscaled_image,folder_path=i['upscaled_folder_path'],resize=False)
                        
            anrp_detections.append(final_json)            
        
    return anrp_detections

In [ ]:
lp_1,lp_2,lp_3=run_ocr(upscaled_image,tr_model,tr_device,tr_processor)
            
final_string=common_chars(lp_1,lp_2,lp_3)
            

In [ ]:
OCR